# General Manufacturing with OpenDXA

This tutorial demonstrates how to use OpenDXA's 2-layer architecture for general manufacturing applications, focusing on production line optimization, quality control, and supply chain management.

## Prerequisites

- Understanding of OpenDXA's 2-layer architecture (Planning and Reasoning layers)
- Familiarity with manufacturing concepts
- Python 3.8 or higher
- OpenDXA package installed

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Create specialized planning strategies for general manufacturing
2. Implement custom reasoning strategies for production optimization
3. Design resource-aware execution strategies
4. Integrate both layers for comprehensive manufacturing solutions

## 1. Setting Up the Environment

First, let's import the necessary components from OpenDXA.

In [ ]:
from opendxa import (
    Plan,
    PlanFactory,
    PlanExecutor,
    Reasoning,
    ReasoningFactory,
    ReasoningExecutor,
    ExecutionContext,
    BaseResource,
    ResourceSelector
)
# Create an execution context
context = ExecutionContext()

## 2. Creating Manufacturing Resources

Let's create specialized resources for general manufacturing:

In [ ]:
class ProductionLineResource(BaseResource):
    """Resource for managing production lines."""
    
    def __init__(self):
        super().__init__(
            name="production_line",
            description="Resource for managing production lines",
            capabilities=["monitor", "optimize", "control"]
        )
    
    async def monitor(self, line_id: str) -> dict:
        """Monitor a production line."""
        return {
            "line_id": line_id,
            "status": "running",
            "metrics": {
                "throughput": 100,
                "efficiency": 0.85,
                "quality_rate": 0.98
            }
        }
    
    async def optimize(self, line_data: dict) -> dict:
        """Optimize production line performance."""
        return {
            "optimization": {
                "status": "optimized",
                "recommendations": [
                    "Adjust line speed",
                    "Reallocate resources"
                ]
            }
        }
    
    async def control(self, command: str) -> dict:
        """Control production line operations."""
        return {
            "control": {
                "command": command,
                "status": "executed",
                "result": "success"
            }
        }

class QualityControlResource(BaseResource):
    """Resource for quality control operations."""
    
    def __init__(self):
        super().__init__(
            name="quality_control",
            description="Resource for quality control operations",
            capabilities=["inspect", "analyze", "report"]
        )
    
    async def inspect(self, product_id: str) -> dict:
        """Inspect a product for quality."""
        return {
            "product_id": product_id,
            "inspection": {
                "status": "passed",
                "defects": [],
                "score": 0.95
            }
        }
    
    async def analyze(self, quality_data: dict) -> dict:
        """Analyze quality control data."""
        return {
            "analysis": {
                "defect_rate": 0.02,
                "trends": [],
                "recommendations": []
            }
        }
    
    async def report(self, period: str) -> dict:
        """Generate quality control report."""
        return {
            "report": {
                "period": period,
                "summary": {
                    "total_inspections": 1000,
                    "defect_rate": 0.02,
                    "improvements": []
                }
            }
        }

# Create and register resources
production_line = ProductionLineResource()
quality_control = QualityControlResource()

context.register_resource(production_line)
context.register_resource(quality_control)

## 3. Creating Planning Strategies

Let's create specialized planning strategies for general manufacturing:

In [ ]:
class ManufacturingPlanningStrategy:
    """Planning strategy for general manufacturing."""
    
    def __init__(self, context: ExecutionContext):
        this.context = context
        this.resource_selector = ResourceSelector(context)
    
    async def plan(self, task: str) -> Plan:
        """Create manufacturing plan."""
        plan = Plan(
            name="manufacturing_plan",
            description=f"Plan for task: {task}",
            objective=task
        )
        
        # Add production line steps
        plan.add_step(PlanStep(
            name="monitor_production",
            description="Monitor production line",
            tool="monitor",
            tool_params={"line_id": "line_001"}
        ))
        
        # Add quality control steps
        plan.add_step(PlanStep(
            name="quality_inspection",
            description="Perform quality inspection",
            tool="inspect",
            tool_params={"product_id": "product_001"}
        ))
        
        return plan

## 4. Creating Reasoning Strategies

Let's create specialized reasoning strategies for general manufacturing:

In [ ]:
class ManufacturingReasoningStrategy:
    """Reasoning strategy for general manufacturing."""
    
    def __init__(self, context: ExecutionContext):
        this.context = context
        this.resource_selector = ResourceSelector(context)
    
    async def reason(self, plan: Plan) -> dict:
        """Execute manufacturing plan."""
        results = {}
        
        # Execute each step
        for step in plan.steps:
            # Select appropriate resource
            resource = await this.resource_selector.select_resource({
                "operation": step.tool,
                "parameters": step.tool_params
            })
            
            # Execute step
            result = await resource.execute(step.tool, step.tool_params)
            results[step.name] = result
            
            # Handle quality issues
            if "inspection" in result and result["inspection"]["status"] != "passed":
                await this._handle_quality_issue(result["inspection"])
        
        return results
    
    async def _handle_quality_issue(self, inspection: dict) -> None:
        """Handle quality control issues."""
        # Example implementation
        print(f"Quality issue detected: {inspection}")

## 5. Putting It All Together

Let's create a complete manufacturing solution:

In [ ]:
# Create planning and reasoning strategies
planning_strategy = ManufacturingPlanningStrategy(context)
reasoning_strategy = ManufacturingReasoningStrategy(context)

# Create a plan for production optimization
plan = await planning_strategy.plan("Optimize production line")

# Execute the plan
results = await reasoning_strategy.reason(plan)

# Print results
print("Manufacturing Process Results:")
for step_name, result in results.items():
    print(f"- {step_name}: {result}")

## 6. Testing the Solution

Let's test our manufacturing solution:

In [ ]:
import pytest

# Test data
TEST_LINE_ID = "line_001"
TEST_PRODUCT_ID = "product_001"

# Test production line monitoring
async def test_production_monitoring():
    result = await production_line.monitor(TEST_LINE_ID)
    assert "line_id" in result
    assert "status" in result
    assert "metrics" in result

# Test quality control
async def test_quality_control():
    result = await quality_control.inspect(TEST_PRODUCT_ID)
    assert "product_id" in result
    assert "inspection" in result
    
    analysis = await quality_control.analyze(result)
    assert "analysis" in analysis
    
    report = await quality_control.report("daily")
    assert "report" in report

# Test planning strategy
async def test_planning_strategy():
    strategy = ManufacturingPlanningStrategy(context)
    plan = await strategy.plan("Test manufacturing task")
    assert plan.name == "manufacturing_plan"
    assert len(plan.steps) > 0

# Test reasoning strategy
async def test_reasoning_strategy():
    strategy = ManufacturingReasoningStrategy(context)
    plan = Plan(
        name="test_plan",
        description="Test plan",
        objective="Test objective"
    )
    
    plan.add_step(PlanStep(
        name="test_step",
        description="Test step",
        tool="monitor",
        tool_params={"line_id": TEST_LINE_ID}
    ))
    
    results = await strategy.reason(plan)
    assert "test_step" in results

# Run the tests
if __name__ == "__main__":
    pytest.main([__file__])

## Next Steps

In this tutorial, we've covered:

1. Setting up the environment for general manufacturing
2. Creating manufacturing resources
3. Creating planning strategies
4. Creating reasoning strategies
5. Putting it all together
6. Testing the solution

In the next tutorial, we'll explore financial applications with OpenDXA.